In [1]:
import pandas as pd
import numpy as np
import warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib
matplotlib.__version__

ModuleNotFoundError: No module named 'matplotlib'

In [86]:
tp_data = pd.read_csv('/Users/tranthong/Desktop/stock-app/state_output/state_output/state_output60.csv')
tp_data.head(5)

# tp_data['test_state_chain'] = tp_data.state_chain.apply(lambda x: len(x))
# tp_data

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,2,1.670342e+09,1.670424e+09,109.218,109.448,NaN,4.121739,-0.704348,3.078261,-0.704348,"[-0.29565217391302895, -0.9913043478261073, -1..."
1,MRK,1,1.670597e+09,1.670598e+09,110.684,110.374,NaN,0.051613,NaN,NaN,-1.000000,NaN
2,MRK,1,1.670602e+09,1.670605e+09,109.916,109.576,NaN,0.305882,NaN,NaN,-1.000000,NaN
3,MRK,1,1.670613e+09,1.670618e+09,109.149,108.764,NaN,0.392208,NaN,NaN,-0.802597,NaN
4,MRK,2,1.670873e+09,1.670873e+09,108.062,108.182,NaN,-0.400000,NaN,NaN,-1.000000,"[0.4, 1.2333333333332543]"


In [91]:
tp_data =[]
print(len(tp_data))

0


In [75]:
def preprocess_tp_data(path):
  tp_data = pd.read_csv(path)
  tp_data.drop_duplicates(inplace = True)
  tp_data.last_r.loc[tp_data.last_r < 0] = -1
  tp_data.state_close_estimate.fillna(tp_data.last_r , inplace = True)
  tp_data.state_max_r.fillna(tp_data.last_r , inplace = True)
  tp_data.state_MA_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_RSI_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_MA_r.loc[tp_data.state_MA_r < -1] = -1
  tp_data.state_max_r.loc[tp_data.state_max_r < -1] = -1
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:str(x).strip('][').split(', '))
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x :[0 - float(element) for element in x])
  return tp_data
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output/state_output/state_output150.csv')

tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].describe()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r
count,528.000000,528.000000,528.000000,528.000000,528.000000
mean,0.671507,2.973849,0.561690,0.076730,0.554948
std,7.564525,8.773564,7.477663,3.814326,7.482398
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-0.041964,-1.000000,-1.000000,-1.000000
50%,-1.000000,0.600000,-1.000000,-1.000000,-1.000000
75%,-1.000000,2.533498,-1.000000,-1.000000,-1.000000
max,83.483268,91.003226,83.483268,59.266667,83.483268


In [101]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[-0.11929824561405766, -0.4701754385964981, -0..."
1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[0.2969696969696954, 0.11515151515150576, -0.7..."
2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[0.09999999999989342, 8.016666666667222, 7.100..."
3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[0.5090909090909526, 0.5090909090909526, 0.327..."
4,MRK,1,1.663338e+09,1.663339e+09,87.218,86.998,-1.000,-0.218182,-1.0,-1.000000,-1.0,NaN


In [76]:
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].sum()

state_close_estimate     354.555719
state_max_r             1570.192064
state_MA_r               296.572213
state_RSI_r               40.513415
last_r                   293.012317
dtype: float64

In [82]:
a = '2022-08-11 16:50:00'
datetime.strptime(a, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2022, 8, 11, 16, 50)

In [42]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output/state_output/*.csv"):
    tp_data = preprocess_tp_data(path)
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)
concat_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[0.11929824561405766, 0.4701754385964981, 0.82..."
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[-0.2969696969696954, -0.11515151515150576, 0...."
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[-0.09999999999989342, -8.016666666667222, -7...."
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[-0.5090909090909526, -0.5090909090909526, -0...."
4,4,MRK,1,1.663338e+09,1.663339e+09,87.218,86.998,-1.000,-0.218182,-1.0,-1.000000,-1.0,NaN


In [62]:
test2 = concat_data.loc[(concat_data.state_chain.isna()) & (concat_data.state_close_estimate > 1)].iloc[4]
get_mt5_raw_data_range(test2.state_ticker, datetime.fromtimestamp(test2.state_day_start), datetime.fromtimestamp(test2.state_day_end))


,index,Open,High,Low,Close,Volume,spread,real_volume,day_num,ticker
0,2022-09-09 13:40:00,733.24,746.60,733.24,745.78,271,82,0,0,REGN
1,2022-09-09 13:45:00,746.37,748.67,741.76,745.08,246,100,0,1,REGN
2,2022-09-09 13:50:00,745.19,748.73,743.65,745.84,218,92,0,2,REGN
3,2022-09-09 13:55:00,745.85,753.74,745.77,751.56,284,85,0,3,REGN
4,2022-09-09 14:00:00,751.56,751.56,742.95,743.44,180,83,0,4,REGN
...,...,...,...,...,...,...,...,...,...,...
68,2022-09-09 19:20:00,722.40,723.27,721.42,723.06,114,45,0,68,REGN
69,2022-09-09 19:25:00,723.06,723.28,722.03,722.18,135,45,0,69,REGN
70,2022-09-09 19:30:00,722.18,722.55,721.28,721.35,70,43,0,70,REGN
71,2022-09-09 19:35:00,721.39,721.44,718.17,719.09,109,40,0,71,REGN


In [50]:
from datetime import datetime
datetime.fromtimestamp(concat_data.loc[(concat_data.state_chain.isna()) ].iloc[1].state_day_start)

datetime.datetime(2022, 9, 17, 2, 10)

In [46]:
concat_data.loc[(concat_data.state_chain.isna()) ].iloc[0].state_day_start

1663338000.0

In [34]:
total_state = len(concat_data)
profit_state = len(concat_data.loc[concat_data.state_close_estimate > 0]) 
loss_state = total_state -  profit_state

total_profit = concat_data.state_close_estimate.sum()
total_win_profit = concat_data.state_close_estimate.loc[concat_data.state_close_estimate > 0].sum()
total_loss_profit = concat_data.state_close_estimate.loc[concat_data.state_close_estimate < 0].sum()
print(total_state , loss_state , profit_state , profit_state/total_state)
print(total_profit,total_loss_profit,total_win_profit)

1046 886 160 0.15296367112810708
325.74728183040054 -885.0 1210.7472818304004


In [35]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data.state_close_estimate.loc[concat_chain_data.state_close_estimate>0].sum()

649.9320075821444

In [30]:
import datetime
from datetime import datetime
test = concat_data.loc[(concat_data.state_chain.isna()) & (concat_data.state_close_estimate > 0)]
test['a'] =  test.state_day_start.apply(lambda x:datetime.fromtimestamp(x) ) 
test['b'] =  test.state_day_end.apply(lambda x:datetime.fromtimestamp(x) ) 
test

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,a,b
9,9,GILD,1,1.662731e+09,1.663076e+09,65.018,64.798,9.816000,14.872727,4.918182,11.645455,4.918182,NaN,2022-09-09 20:50:00,2022-09-13 20:40:00
11,11,PYPL,1,1.662559e+09,1.662736e+09,92.366,91.826,6.507407,9.451852,6.507407,5.729630,6.507407,NaN,2022-09-07 21:00:00,2022-09-09 22:10:00
30,30,PFE,1,1.662646e+09,1.663073e+09,46.234,46.024,4.504762,9.600000,4.504762,4.504762,4.504762,NaN,2022-09-08 21:00:00,2022-09-13 19:45:00
39,39,MMM,1,1.662644e+09,1.662653e+09,117.694,117.084,0.780328,2.862295,0.698361,0.780328,0.780328,NaN,2022-09-08 20:40:00,2022-09-08 23:05:00
55,55,TMO,1,1.662573e+09,1.662644e+09,551.756,551.116,2.756250,3.084375,2.756250,2.756250,2.756250,NaN,2022-09-08 00:45:00,2022-09-08 20:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,144,ADP,1,1.668537e+09,1.668696e+09,249.392,248.462,2.363441,6.954839,0.406452,2.363441,2.363441,NaN,2022-11-16 01:25:00,2022-11-17 21:40:00
638,147,BMY,1,1.667832e+09,1.668026e+09,78.821,78.606,5.065116,10.506977,5.065116,1.995349,5.065116,NaN,2022-11-07 21:45:00,2022-11-10 03:35:00
639,148,BMY,1,1.668091e+09,1.668094e+09,79.483,79.288,1.369231,4.907692,2.651282,1.369231,1.369231,NaN,2022-11-10 21:40:00,2022-11-10 22:35:00
641,150,BMY,1,1.668786e+09,1.669733e+09,77.198,76.903,3.972881,8.142373,3.972881,3.227119,3.972881,NaN,2022-11-18 22:35:00,2022-11-29 21:50:00


In [32]:
test['a2'] = test.a.apply(lambda x : x.timestamp())
#test.loc[test.a2 == test.state_day_start]
test['a2_1'] = test['a2'].apply(lambda x:datetime.fromtimestamp(x) )
test

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,a,b,a2,a2_1
9,9,GILD,1,1.662731e+09,1.663076e+09,65.018,64.798,9.816000,14.872727,4.918182,11.645455,4.918182,NaN,2022-09-09 20:50:00,2022-09-13 20:40:00,1.662757e+09,2022-09-10 03:50:00
11,11,PYPL,1,1.662559e+09,1.662736e+09,92.366,91.826,6.507407,9.451852,6.507407,5.729630,6.507407,NaN,2022-09-07 21:00:00,2022-09-09 22:10:00,1.662584e+09,2022-09-08 04:00:00
30,30,PFE,1,1.662646e+09,1.663073e+09,46.234,46.024,4.504762,9.600000,4.504762,4.504762,4.504762,NaN,2022-09-08 21:00:00,2022-09-13 19:45:00,1.662671e+09,2022-09-09 04:00:00
39,39,MMM,1,1.662644e+09,1.662653e+09,117.694,117.084,0.780328,2.862295,0.698361,0.780328,0.780328,NaN,2022-09-08 20:40:00,2022-09-08 23:05:00,1.662670e+09,2022-09-09 03:40:00
55,55,TMO,1,1.662573e+09,1.662644e+09,551.756,551.116,2.756250,3.084375,2.756250,2.756250,2.756250,NaN,2022-09-08 00:45:00,2022-09-08 20:40:00,1.662598e+09,2022-09-08 07:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,144,ADP,1,1.668537e+09,1.668696e+09,249.392,248.462,2.363441,6.954839,0.406452,2.363441,2.363441,NaN,2022-11-16 01:25:00,2022-11-17 21:40:00,1.668562e+09,2022-11-16 08:25:00
638,147,BMY,1,1.667832e+09,1.668026e+09,78.821,78.606,5.065116,10.506977,5.065116,1.995349,5.065116,NaN,2022-11-07 21:45:00,2022-11-10 03:35:00,1.667858e+09,2022-11-08 04:45:00
639,148,BMY,1,1.668091e+09,1.668094e+09,79.483,79.288,1.369231,4.907692,2.651282,1.369231,1.369231,NaN,2022-11-10 21:40:00,2022-11-10 22:35:00,1.668116e+09,2022-11-11 04:40:00
641,150,BMY,1,1.668786e+09,1.669733e+09,77.198,76.903,3.972881,8.142373,3.972881,3.227119,3.972881,NaN,2022-11-18 22:35:00,2022-11-29 21:50:00,1.668811e+09,2022-11-19 05:35:00


In [56]:
test = concat_data.loc[(concat_data.state_chain.notna()) & (concat_data.state_close_estimate > 0)]
test['a'] =  test.state_day_start.apply(lambda x:datetime.fromtimestamp(x) ) 
test['b'] =  test.state_day_end.apply(lambda x:datetime.fromtimestamp(x) ) 
def get_mt5_raw_data_range(ticker, start, end):
    stock_dir = '/Users/tranthong/Desktop/stock-app/stock_data_M5/'+ ticker +'_2022.csv'
    rates =  pd.read_csv(stock_dir, index_col =[0])
    rates['index'] = rates['index'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    a = rates.loc[ (rates['index'] > start) & (rates['index'] < end )]
    a.reset_index(inplace = True ,drop = True)
    a['day_num'] = a.index
    return a

row = test.iloc[1]
test_data = get_mt5_raw_data_range(row.state_ticker, row.a, row.b)
test_data['test'] = ( test_data.Close - row.state_open) / abs(row.state_open - row.state_stop_loss) if row.state_order == 1 else (- test_data.Close + row.state_open) / abs(row.state_open - row.state_stop_loss)
test_data.test , row.state_chain


(0      0.447222
 1      0.308333
 2      0.100000
 3      0.141667
 4      0.183333
          ...   
 229    2.461111
 230    2.433333
 231    2.641667
 232    2.586111
 233    2.877778
 Name: test, Length: 234, dtype: float64,
 '[0.28888888888888714, 0.03888888888887727, -0.10000000000000395, -0.08611111111112767, 0.0944444444444416, 0.0944444444444416, -0.4055555555555584, -0.39166666666668215, -0.7111111111111326, -0.7527777777777812, -0.5305555555555633, -0.46111111111112274, -1.1694444444444643, -1.0305555555555634, -0.6972222222222366, -0.8083333333333454, -0.46111111111112274, -0.6694444444444643, -0.9055555555555781, -0.3500000000000138, -0.6833333333333406, -0.5722222222222316, -0.8777777777777861, -0.5583333333333553, -0.6555555555555683, -0.7388888888889049, -0.3222222222222415, 0.14999999999998617, 0.511111111111105, 0.4277777777777684, 0.372222222222204, 0.372222222222204, 0.03888888888887727, -0.3083333333333455, -0.5722222222222316, -0.2805555555555732, 0.01111111111110

In [266]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data['len_state_chain']  = concat_chain_data.state_chain.apply(lambda x : len(x))
concat_chain_data = concat_chain_data.loc[concat_chain_data.len_state_chain > 2]
concat_chain_data.head(50)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,len_state_chain
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000000,-0.119298,-1.000000,-1.000000,-1.000000,"[-0.11929824561405766, -0.4701754385964981, -0...",4
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000000,0.296970,-1.000000,-1.000000,-1.000000,"[0.2969696969696954, 0.11515151515150576, -0.7...",4
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536000,17.266667,3.100000,8.016667,3.100000,"[0.09999999999989342, 8.016666666667222, 7.100...",77
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000000,0.509091,-1.000000,-1.000000,-1.000000,"[0.5090909090909526, 0.5090909090909526, 0.327...",9
7,7,MRK,2,1.663696e+09,1.663702e+09,85.906,86.016,-1.000000,2.236364,-1.000000,-1.000000,-1.000000,"[0.32727272727274137, 1.3272727272727414, 2.14...",24
10,10,GILD,2,1.664211e+09,1.664215e+09,62.448,62.528,-1.000000,3.350000,-1.000000,-1.000000,-1.000000,"[0.8499999999999911, 1.9750000000000578, 2.225...",14
18,18,PYPL,2,1.663248e+09,1.663249e+09,95.902,96.072,-1.000000,5.482353,-1.000000,5.482353,-1.000000,"[0.4235294117647177, 5.482352941176428, -2.929...",3
19,19,PYPL,2,1.663349e+09,1.663351e+09,92.894,93.184,-1.000000,0.393103,-1.000000,-1.000000,-1.000000,"[0.3931034482758685, -0.6758620689654956, -0.1...",7
23,23,PYPL,2,1.664458e+09,1.664803e+09,89.472,90.192,2.780556,5.516667,2.780556,2.780556,2.780556,"[-0.28888888888888714, -0.03888888888887727, 0...",233
26,26,IBM,2,1.663336e+09,1.663337e+09,124.558,124.988,-1.000000,-0.027907,-1.000000,-1.000000,-1.000000,"[-0.40000000000003305, -0.027906976744187583, ...",5


In [264]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data['len_state_chain']  = concat_chain_data.state_chain.apply(lambda x : len(x))
concat_chain_data.loc[(concat_chain_data.len_state_chain <= 2) & (concat_chain_data.state_close_estimate > 0)].state_close_estimate.sum() 

4.446544523588392

In [237]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

concat_chain_data['estimate_with_thresh'] = concat_chain_data.state_chain.apply(lambda x : get_return_with_max_estimate(x) )


In [242]:
concat_chain_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,len_state_chain,estimate_with_thresh
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[-0.11929824561405766, -0.4701754385964981, -0...",4,-1.000000
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[0.2969696969696954, 0.11515151515150576, -0.7...",4,-1.000000
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[0.09999999999989342, 8.016666666667222, 7.100...",77,17.266667
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[0.5090909090909526, 0.5090909090909526, 0.327...",9,-1.000000
7,7,MRK,2,1.663696e+09,1.663702e+09,85.906,86.016,-1.000,2.236364,-1.0,-1.000000,-1.0,"[0.32727272727274137, 1.3272727272727414, 2.14...",24,-1.000000


In [241]:
concat_chain_data['estimate_with_thresh'] = concat_chain_data.state_chain.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
concat_chain_data.estimate_with_thresh.sum()

-27.32915049847987

In [78]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output/state_output/*.csv"):
    tp_data = preprocess_tp_data(path)
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)
concat_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[-0.11929824561405766, -0.4701754385964981, -0..."
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[0.2969696969696954, 0.11515151515150576, -0.7..."
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[0.09999999999989342, 8.016666666667222, 7.100..."
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[0.5090909090909526, 0.5090909090909526, 0.327..."
4,4,MRK,1,1.663338e+09,1.663339e+09,87.218,86.998,-1.000,-0.218182,-1.0,-1.000000,-1.0,NaN


In [81]:
tp_data = concat_data
stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].groupby('state_ticker').sum()
stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
stock_tp_data.win_count.fillna(0 , inplace = True)
stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 
stock_tp_data.loc[stock_tp_data.state_close_estimate > -20].state_close_estimate.sum()

463.97348031150364